In [1]:
import nltk
import pandas as pd
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# importing the requests library
import requests
import pickle
import re

In [2]:
df = pd.read_csv('./silver_dataset.csv')
print(df)

       Unnamed: 0                   id      conversation_id  \
0               0  1376962511099035652  1376962511099035652   
1               1  1376961972885938181  1376961972885938181   
2               2  1376961945153196032  1376961945153196032   
3               3  1376960959932153863  1376960959932153863   
4               4  1376959717151449090  1376959717151449090   
...           ...                  ...                  ...   
14343       14343  1344720754512945154  1344720754512945154   
14344       14344  1344719980370268160  1344719980370268160   
14345       14345  1344718465551241218  1344718465551241218   
14346       14346  1344715298822352896  1344715298822352896   
14347       14347  1344715020769214464  1344715020769214464   

                created_at        date      time  timezone  \
0      2021-03-30 23:50:20  2021-03-30  23:50:20       530   
1      2021-03-30 23:48:11  2021-03-30  23:48:11       530   
2      2021-03-30 23:48:05  2021-03-30  23:48:05       53

In [3]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-12-28 16:04:16,552 loading file /home/mtnshh/.flair/models/sentiment-en-mix-distillbert_4.pt


In [4]:
def comp_score_flair(text):
    s = flair.data.Sentence(text)
    flair_sentiment.predict(s)
    total_sentiment = s.labels
    return str(total_sentiment[0]).split(' ')[0]

def score_flair(text):
    s = flair.data.Sentence(text)
    flair_sentiment.predict(s)
    total_sentiment = s.labels
    temp = str(total_sentiment[0]).split(' ')[0]
    if temp == "POSITIVE":
        return float(str(total_sentiment[0]).split(' ')[1][1:-1])
    else:
        return -float(str(total_sentiment[0]).split(' ')[1][1:-1])

df['score_flair'] = df['tweet'].apply(score_flair)    
df['comp_score_flair'] = df['tweet'].apply(comp_score_flair)

In [5]:
df

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,score_flair,comp_score_flair
0,0,1376962511099035652,1376962511099035652,2021-03-30 23:50:20,2021-03-30,23:50:20,530,144367359,nowtheendbegins,Now The End Begins,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,-0.8668,NEGATIVE
1,1,1376961972885938181,1376961972885938181,2021-03-30 23:48:11,2021-03-30,23:48:11,530,397967759,babydollirish2,I💚Dublin,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,-0.8309,NEGATIVE
2,2,1376961945153196032,1376961945153196032,2021-03-30 23:48:05,2021-03-30,23:48:05,530,3264486358,ioannadementi,IoannaDementiEU,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,0.5216,POSITIVE
3,3,1376960959932153863,1376960959932153863,2021-03-30 23:44:10,2021-03-30,23:44:10,530,18414632,chrisbanks,Chris Banks,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,0.9875,POSITIVE
4,4,1376959717151449090,1376959717151449090,2021-03-30 23:39:13,2021-03-30,23:39:13,530,1428131408,sarahmwc,Sarah Coles,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,0.9966,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14343,14343,1344720754512945154,1344720754512945154,2021-01-01 00:33:06,2021-01-01,00:33:06,530,997310482179678214,rkh_md,"Ryan K Harrison, MD",...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,0.9863,POSITIVE
14344,14344,1344719980370268160,1344719980370268160,2021-01-01 00:30:01,2021-01-01,00:30:01,530,1164940374374330368,chinascience,China Science,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,-0.9964,NEGATIVE
14345,14345,1344718465551241218,1344718465551241218,2021-01-01 00:24:00,2021-01-01,00:24:00,530,393776618,natalie_zink,Natalie Zink,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,0.9571,POSITIVE
14346,14346,1344715298822352896,1344715298822352896,2021-01-01 00:11:25,2021-01-01,00:11:25,530,2725149830,ujjram,Ujjwal R. MD/CPE/MBA/MPE,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,0.9857,POSITIVE


In [6]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [7]:
from textblob import TextBlob

In [8]:
df['scores'] = df['tweet'].apply(lambda review: sid.polarity_scores(review))

In [9]:
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

In [10]:
df['comp_score'] = df['compound'].apply(lambda c: 'POSITIVE' if c >=0 else 'NEGATIVE')

In [11]:
df['scores_text_blob'] = df['tweet'].apply(lambda review: TextBlob(review).sentiment.polarity)
df['comp_score_text_blob'] = df['scores_text_blob'].apply(lambda c: 'POSITIVE' if c >=0 else 'NEGATIVE')

In [12]:
df

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,translate,trans_src,trans_dest,score_flair,comp_score_flair,scores,compound,comp_score,scores_text_blob,comp_score_text_blob
0,0,1376962511099035652,1376962511099035652,2021-03-30 23:50:20,2021-03-30,23:50:20,530,144367359,nowtheendbegins,Now The End Begins,...,NaN,NaN,NaN,-0.8668,NEGATIVE,"{'neg': 0.053, 'neu': 0.947, 'pos': 0.0, 'comp...",-0.1280,NEGATIVE,0.000000,POSITIVE
1,1,1376961972885938181,1376961972885938181,2021-03-30 23:48:11,2021-03-30,23:48:11,530,397967759,babydollirish2,I💚Dublin,...,NaN,NaN,NaN,-0.8309,NEGATIVE,"{'neg': 0.127, 'neu': 0.873, 'pos': 0.0, 'comp...",-0.2901,NEGATIVE,0.000000,POSITIVE
2,2,1376961945153196032,1376961945153196032,2021-03-30 23:48:05,2021-03-30,23:48:05,530,3264486358,ioannadementi,IoannaDementiEU,...,NaN,NaN,NaN,0.5216,POSITIVE,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,POSITIVE,0.000000,POSITIVE
3,3,1376960959932153863,1376960959932153863,2021-03-30 23:44:10,2021-03-30,23:44:10,530,18414632,chrisbanks,Chris Banks,...,NaN,NaN,NaN,0.9875,POSITIVE,"{'neg': 0.0, 'neu': 0.81, 'pos': 0.19, 'compou...",0.8020,POSITIVE,-0.088889,NEGATIVE
4,4,1376959717151449090,1376959717151449090,2021-03-30 23:39:13,2021-03-30,23:39:13,530,1428131408,sarahmwc,Sarah Coles,...,NaN,NaN,NaN,0.9966,POSITIVE,"{'neg': 0.063, 'neu': 0.631, 'pos': 0.306, 'co...",0.8074,POSITIVE,0.400000,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14343,14343,1344720754512945154,1344720754512945154,2021-01-01 00:33:06,2021-01-01,00:33:06,530,997310482179678214,rkh_md,"Ryan K Harrison, MD",...,NaN,NaN,NaN,0.9863,POSITIVE,"{'neg': 0.0, 'neu': 0.934, 'pos': 0.066, 'comp...",0.5399,POSITIVE,0.246250,POSITIVE
14344,14344,1344719980370268160,1344719980370268160,2021-01-01 00:30:01,2021-01-01,00:30:01,530,1164940374374330368,chinascience,China Science,...,NaN,NaN,NaN,-0.9964,NEGATIVE,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,POSITIVE,0.100000,POSITIVE
14345,14345,1344718465551241218,1344718465551241218,2021-01-01 00:24:00,2021-01-01,00:24:00,530,393776618,natalie_zink,Natalie Zink,...,NaN,NaN,NaN,0.9571,POSITIVE,"{'neg': 0.032, 'neu': 0.775, 'pos': 0.193, 'co...",0.7947,POSITIVE,0.100000,POSITIVE
14346,14346,1344715298822352896,1344715298822352896,2021-01-01 00:11:25,2021-01-01,00:11:25,530,2725149830,ujjram,Ujjwal R. MD/CPE/MBA/MPE,...,NaN,NaN,NaN,0.9857,POSITIVE,"{'neg': 0.0, 'neu': 0.793, 'pos': 0.207, 'comp...",0.8718,POSITIVE,0.500000,POSITIVE
